# "Visualize" Building Numbers (because we can!)

This notebook uses the free Microsoft "US Builing Footprints" [dataset](https://github.com/Microsoft/USBuildingFootprints), enriched with [HERE](https://here.com) geolocation information and loaded from a public HERE [Data Hub](https://here.xyz) "space". It searches footprints (GeoJSON features) up to 2 km away from any selected location and colors them according to their house numbers given color scale for all numbers in this area. While the practical use is limited a certain grid structure can be observed that will likely look less rigid outside the US. Something very similar can be done to show building footprint area _sizes_ for example.

This code uses a new Python package named "xyzspaces" (see [GitHub repo](https://github.com/heremaps/xyz-spaces-python)) providing access to the Open Source XYZ technology at the core of the Data Hub installation at HERE. A registration to the HERE [Data Hub](https://here.xyz) and basic use is free up to a certain limit, but some features require a "pro" plan.

In [ ]:
import os
import re
from functools import partial
from io import StringIO

import geojson
import bqplot.pyplot as plt
from branca.colormap import LinearColormap
from ipywidgets import Button, HTML, Layout
from ipyleaflet import (basemaps, FullScreenControl, GeoJSON, LayersControl,
    LegendControl, Map, Marker, Popup, SearchControl, ScaleControl, WidgetControl)

import xyzspaces
# The following needs an "XYZ_TOKEN" environment variable:
from xyzspaces.datasets import get_microsoft_buildings_space as msbf

In [ ]:
def house_color(feat, cs):
    try:
        return cs(int(feat["properties"]["estimatedHouseNumber"]))
    except:
        return "#aaaaaa"

In [ ]:
def hist(data):
    fig = plt.figure(fig_margin=dict(top=5, bottom=35, left=30, right=15))
    hist = plt.hist(data)
    fig.layout.width = "200px"
    fig.layout.height = "150px"
    hist.bins = 20
    if data:
        vmin, vmax = min(data), max(data)
        cs = LinearColormap(["green", "blue"], vmin=vmin, vmax=vmax)
        hist.colors = [cs(vmin + i * (vmax - vmin)/hist.bins) for i in range(hist.bins)]
    vx, hx = fig.axes
    vx.num_ticks = 4
    hx.num_ticks = 5
    hx.label = "Histogram Bldg. Numbers"
    return fig

In [ ]:
def marker_moved(event, mymap=None, mylegend=None, myhist=None):
    if event["name"] not in ["location", "visible"]:
        return
    mymap.layers = tuple([l for l in mymap.layers if type(l) != GeoJSON])
    lat, lon = event["new"] if event["name"] == "location" else event["owner"].location
    features = list(msbf().spatial_search(lat=lat, lon=lon, radius=2000))
    house_nums = [int(feat["properties"]["estimatedHouseNumber"]) 
        for feat in features if "estimatedHouseNumber" in feat["properties"] \
             and re.match("^\d+$", feat["properties"]["estimatedHouseNumber"])]
    vmin, vmax = min(house_nums), max(house_nums)
    cs = LinearColormap(["green", "blue"], vmin=vmin, vmax=vmax)

    data = geojson.FeatureCollection(features=features)
    gj = GeoJSON(data=data, name="Building numbers",
        hover_style={"weight": 4, 'fillOpacity': 0.6},
        style_callback=lambda feat: {"weight": 2, "color": house_color(feat, cs)})
    mymap += gj

    myhist.widget = hist(house_nums)
    if myhist not in mymap.controls:
        mymap += myhist

    mylegend.legend = {f"Max: {vmax}": "blue", f"Min: {vmin}": "green", "Unknown": "#aaaaaa"}
    if mylegend not in mymap.controls:
        mymap += mylegend
    event["owner"].popup = HTML(
        f"<b>Lat/Lon:</b> {lat}/{lon},<br/><b>Radius:</b> 2000 m, <b>#Features:</b> {len(features)}</center>")

In [ ]:
def add_marker(button, mymap=None, mylegend=None, myhist=None):
    mymap.layers = tuple([l for l in mymap.layers if type(l) not in [Marker, GeoJSON]])
    mk = Marker(location=mymap.center, name="Center", visible=False)
    mk.observe(partial(marker_moved, mymap=mymap, mylegend=mylegend, myhist=myhist))
    mymap += mk
    mk.visible = True

In [ ]:
lat, lon = 38.89759, -77.03665
m = Map(center=[lat, lon], zoom=14, basemap=basemaps.CartoDB.Positron)
m.layout.width = "1000px"
m.layout.height = "600px"
m += FullScreenControl(position="topleft")
m += LayersControl(position="topleft")
m += ScaleControl(position="bottomleft")
legend = {"Max": "blue", "Min": "green", "Unknown": "#aaaaaa"}
lg = LegendControl(legend=legend, name="Bldg. numbers", position="topright")

hfig = HTML()
hi = WidgetControl(name="Histogramm", widget=hfig, position="bottomright")

btn = Button(icon="plus-circle", tooltip="Add at center", layout=Layout(width="30px"))
btn.on_click(partial(add_marker, mymap=m, mylegend=lg, myhist=hi))
m += WidgetControl(widget=btn, position="topleft")

m += SearchControl(auto_type=True, auto_collapse=True, position="topleft",
    url='https://nominatim.openstreetmap.org/search?format=json&q={s}')

In [ ]:
m